## The Money Control Project -  Part 2

______________________________________________________________________________________________________________________________
In part 1, I had scrapped the data from the moneycontrol website and arranged the required information in a tabular form. The information we had collected were:

    Date  |  Stock  |  Action  |  Target  | Recommender
  ______________________________________________________


In this part we will extend this data further by scrapping data from the NSE website. We will add three more columns to this existing table. They will be,

    Target Met | Date of Target Met  |  High  | Date to Nearest
   ________________________________________________________________


Variable Descriptions:

    Date              : Date of a recommendation
    Stock             : Stock bout which the recommendations were made
    Action            : Buy, sell, hold, accumulate decisions
    Target            : Target of the stock in Rupees
    Recommender       : Who made this recommendation?
    Target Met        : Whether the suggested target was met within 1 year from the date of recommendation
    Date of Target Met: Date when the target is met
    High              : Actual price that crosses the target for the first time or The nearest value, if the target is not met
    Date to Nearest   : The date corresponding to the column 'Nearest'

________________________________________________________

Note: Part 2 is not very immediate of part one. I had faced some challenges in between. The Stock names were not the same throughout. For example, State Bank of India is written as SBI, or SBIN or State Bank of India. So we have replaced the stock names by the NSE stock codes.
_______________________________________________________________________________-

## Libraries
__________________________________________________________________________________________________________________________-

In [1]:
#importing nse library to get nse stock data
from nsepy import get_history

In [2]:
import pandas as pd
from datetime import date
import os

## Data
_____________________________________________________________________________________________________________________

In [3]:
#reading the scrapped data 
money=pd.read_excel("C:\\Users\\Gourab\\Downloads\\MoneyControl_Scrapped data.xlsx")

In [4]:
money.head()

,Date,Stock_Name,New Stock Name,NSE DESC,NSE CODE,Action,Target,Recommender
0,"jul 28, 2017",canara bank,CANARA BANK,CANARA BANK LTD,CANBK,hold,348,reliance securities
1,"jul 21, 2017",canara bank,CANARA BANK,CANARA BANK LTD,CANBK,Buy,377,kr choksey
2,"may 16, 2017",canara bank,CANARA BANK,CANARA BANK LTD,CANBK,hold,390,axisÂ direct
3,"jan 23, 2017",canara bank,CANARA BANK,CANARA BANK LTD,CANBK,Buy,350,kr choksey
4,"jan 23, 2017",canara bank,CANARA BANK,CANARA BANK LTD,CANBK,sell,200,religare


## Basic Editing
_____________________________________________________________________________________________________________________

In [5]:
money['Action'] = money.Action.str.upper()               #Converting the Action column to upper case
money['Recommender'] = money.Recommender.str.upper()     #Converting the Recommender column to upper case
money['Date'] = money.Date.str.strip().replace(",", '')  #Stripping off white spaces from the Date column and removing the comma

In [6]:
money = money.drop(['Stock_Name','New Stock Name'],axis=1) #Deleting unnecessary columns

In [7]:
money.head()

,Date,NSE DESC,NSE CODE,Action,Target,Recommender
0,"jul 28, 2017",CANARA BANK LTD,CANBK,HOLD,348,RELIANCE SECURITIES
1,"jul 21, 2017",CANARA BANK LTD,CANBK,BUY,377,KR CHOKSEY
2,"may 16, 2017",CANARA BANK LTD,CANBK,HOLD,390,AXISÂ DIRECT
3,"jan 23, 2017",CANARA BANK LTD,CANBK,BUY,350,KR CHOKSEY
4,"jan 23, 2017",CANARA BANK LTD,CANBK,SELL,200,RELIGARE


________________________________________________________________________________________________________________________________

## Scrapping Data from NSE India Website
____________________________________________________________________________________________________________________________-

### Part A: Understanding how the nsepy.get_history() function works

In [8]:
#Considering the first row.
d =money.Date.iloc[0]   #Date of row 1
d = pd.to_datetime(d)   #Converting the string to a datetime object

In [9]:
startDate = d.date()         #Gives the date
startYear = d.year           #Gives the year
day       = d.day            #Gives the day
month     = d.month          #Gives the month
endYear   = startYear + 1    #The end date (since we are checking only for 365 days)

print('Start Year\t: ', startYear, "\n",
     'Day\t\t: ', day, "\n",
     'Month\t\t: ', month, "\n",
     "End Year\t: ", endYear)

Start Year	:  2017 
 Day		:  28 
 Month		:  7 
 End Year	:  2018


#### The get_history() function

In [10]:
#The get_history() function in the nsepy package
#This function will help us to get the stock performance data of a particular stock between two dates

hist = get_history(symbol='CANBK', start=date(startYear, month, day), end=date(endYear, month, day))

In [11]:
#The first 3 lines of the data
hist.head(3)

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2017-07-28,CANBK,EQ,353.95,355.0,364.80,350.55,364.35,364.0,359.61,2471191,8.886597e+13,21637,685695,0.2775
2017-07-31,CANBK,EQ,364.00,362.0,369.95,360.85,367.85,368.1,366.71,2322450,8.516544e+13,18750,512639,0.2207
2017-08-01,CANBK,EQ,368.10,369.8,370.45,361.05,363.50,362.7,364.04,2017458,7.344340e+13,15114,656505,0.3254


In [12]:
#the last two lines of the data
hist.tail(2)

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2018-07-26,CANBK,EQ,258.95,268.00,281.3,262.7,279.95,278.90,272.35,24218706,6.596008e+14,149931,2120780,0.0876
2018-07-27,CANBK,EQ,278.90,278.25,282.5,273.7,279.25,279.25,278.29,8504426,2.366736e+14,58905,343664,0.0404


For our purpose, we will not need all these columns. We will be particularly need only one column, which is the 'High' column. With the recommended target we will compare the high column for 1 year and check if the target is met.

### Extending the Dataset 

    For i in range(0,len(money)):
    
        Get the date from the Date column
        Extract the startYear, month and day
        endYear   = start year + 1
        startDate = date(startYear, month, day)
        endDate   = date(endYear,   month, day)

        high = Get the 'High' values of the stock between the startDate and the endDate using get_history() function

        if any 'High' value exceeds the 'Target':
            money['Target_Met'][i]         = 1
            money['Date_of_Target_Met'][i] = ...
            money['Nearest'][i]            = None
            money['Date_to_Nearest'][i]    = ...

        else:
            money['Target_Met'][i]         = 0
            money['Date_of_Target_Met'][i] = ''
            money['Nearest']               = max(High)
            money['Date_to_Nearest'][i]    = None



In [22]:
money['Target_Met']         = None
money['Date_of_Target_Met'] = None
money['High']               = None
money['Date_to_Nearest']    = None

In [23]:
for i in range(0,20):
    d         = money.Date[i]
    d         = pd.to_datetime(d)
    d         = d.date()
    startYear = d.year
    month     = d.month
    day       = d.day
    endYear   = startYear + 1

    #startDate = date(startYear, month, day)
    #endDate   = date(endYear, month, day)
    #print('something')

    high = get_history(money['NSE CODE'][i], start=date(startYear, month, day), end=date(endYear, month, day))['High']
    
    #if any 'High' value exceeds the 'Target':
    if sum(high > money.Target[i]) > 1:
            money['Target_Met'][i] = 1
            money['High'][i] = high[list(high > money.Target[i]).index(True)]
            money['Date_of_Target_Met'][i] = str(high.index[list(high > money.Target[i]).index(True)])
            money['Nearest'][i] = ''
            money['Date_to_Nearest'][i] = ''

    else:
            money['Target_Met'][i]      = 0
            money['High'][i] = max(high)
            money['Date_of_Target_Met'][i] = ''
            money['Date_to_Nearest'][i] = str(high.index[list(high).index(max(high))])
            


In [20]:
money.head(20)

,Date,NSE DESC,NSE CODE,Action,Target,Recommender,Target_Met,Date_of_Target_Met,Nearest,Date_to_Nearest,High
0,"jul 28, 2017",CANARA BANK LTD,CANBK,HOLD,348,RELIANCE SECURITIES,1,2017-07-28,None,,364.8
1,"jul 21, 2017",CANARA BANK LTD,CANBK,BUY,377,KR CHOKSEY,1,2017-10-25,None,,444.1
2,"may 16, 2017",CANARA BANK LTD,CANBK,HOLD,390,AXISÂ DIRECT,1,2017-10-25,None,,444.1
3,"jan 23, 2017",CANARA BANK LTD,CANBK,BUY,350,KR CHOKSEY,1,2017-04-28,None,,361.7
4,"jan 23, 2017",CANARA BANK LTD,CANBK,SELL,200,RELIGARE,1,2017-01-23,None,,276.45
5,"nov 18, 2016",CANARA BANK LTD,CANBK,HOLD,246,SPA RESEARCH,1,2016-11-18,None,,329.8
6,"jul 29, 2016",CANARA BANK LTD,CANBK,SELL,190,RELIGARE,1,2016-07-29,None,,258.8
7,"jul 27, 2016",CANARA BANK LTD,CANBK,BUY,300,AXIS DIRECT,1,2016-09-07,None,,314.5
8,"may 30, 2016",CANARA BANK LTD,CANBK,BUY,201,RELIANCE SECURITIES,1,2016-05-31,None,,208.9
9,"feb 26, 2016",CANARA BANK LTD,CANBK,HOLD,177,KRCHOKSEY,1,2016-03-02,None,,179.05


_________________________________________________________________________________________________________________________-
In the final data, I have created another columns:

    Days to target met

Which is the number of Days to meet the target.

In [ ]:
#Creating the column 'Days to Target Met'

In [ ]:
#Converting the column in datetime format
money['Date'] = pd.to_datetime(money.Date)
money['Date_of_Target_Met'] = pd.to_datetime(money.Date_of_Target_Met)

money.iloc[0:20, ]

In [ ]:
import numpy as np
money2['Days_to_Target'] = None

DaysToTarget = []

for i in range(0,20):
    if money2.Date_of_Target_Met[i] is not pd.NaT:
        DaysToTarget.append((money2.Date_of_Target_Met[i].date() - money2.Date[i].date()).days)  #Date difference
    else:
        DaysToTarget.append('')

money2['Days_to_Target'] = DaysToTarget

In [ ]:
money.iloc[0:20, ]

____________________________________________________________________________________________________________________________-
And our data is ready for further analysis...